In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, AutoModelForSequenceClassification
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [3]:
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

# Set the default device for tensors
torch.cuda.set_device(0)  # Set the GPU you want to use if you have multiple GPUs

Using NVIDIA GeForce GTX 1650


## 1 - Dataset and LLM

In [4]:
dataset = load_dataset("imdb")

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [9]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
original_tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
def tokenize_function(examples):
    start_prompt = 'Give sentiment positive or negative for the following movie review.\n\n'
    end_prompt = '\n\nSentiment:: '

    # Convert labels to text
    label_map = {0: "negative", 1: "positive"}
    labels = []
    for label in examples["label"]:
        if label in label_map:
            labels.append(label_map[label])
        else:
            labels.append("unknown")

    # Construct prompts and tokenize
    prompts = [start_prompt + text + end_prompt for text in examples["text"]]
    tokenized_inputs = original_tokenizer(prompts, padding="max_length", truncation=True, return_tensors="pt")
    tokenized_labels = original_tokenizer(labels, padding="max_length", truncation=True, return_tensors="pt")

    # Return the processed batch
    return {"input_ids": tokenized_inputs.input_ids, "labels": tokenized_labels.input_ids, "labeled": labels}

In [11]:
# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [12]:
# Filter every 100th example
filtered_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [13]:
filtered_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'labels', 'labeled'],
        num_rows: 250
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'labels', 'labeled'],
        num_rows: 250
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'labels', 'labeled'],
        num_rows: 500
    })
})

In [5]:
# Function to clean text from HTML tags
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    # Remove escape characters
    text = re.sub(r'\\', '', text)
    return text

In [6]:
# Clean the text in the dataset
for split in ['train', 'test', 'unsupervised']:
    dataset[split] = dataset[split].map(lambda example: {'text': clean_text(example['text']), 'label': example['label']})

In [8]:
from transformers import pipeline

pipe = pipeline(task="text2text-generation", model=original_model, tokenizer=original_tokenizer, device=device)

In [14]:
def calculate_summary(input_text):
    prompt = "Summarize the following movie review.: " + input_text + "\n\nSentiment:"
    output = pipe(prompt, max_length=512)
    return output[0]['generated_text']

In [13]:
input_text = original_tokenizer.decode(filtered_datasets['test']['input_ids'], skip_special_tokens=True)
print(input_text)
# prediction = calculate_sentiment(input_text)

TypeError: argument 'ids': 'list' object cannot be interpreted as an integer

In [7]:
index = 200

text = dataset['test']['text'][index]
label = dataset['test']['label'][index]

prompt = f"""
Summarize the following movie review.

{text}

Summary: """

inputs = original_tokenizer(prompt, return_tensors='pt', max_length=512, truncation=True)
output = original_tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

# Convert the label to "negative" if 0, and "positive" if 1
sentiment_label = "positive" if label == 1 else "negative"

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE IMDB SENTIMENT:\n{sentiment_label}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following movie review.

He who fights with monsters might take care lest he thereby become a monster. And if you gaze for long into an abyss, the abyss gazes also into you.<br /><br />Yes, this is from Nietzsche's Aphorism 146 from "Beyond Good and Evil". And that's what you find at the start of this movie.<br /><br />If you watch the whole movie, you will doubt if it was the message that the Ram Gopal Varma Production wanted to pass on. As the scenes crop up one by one, quite violent and at times puke-raking, the viewer is expected to forget the Nietzsche quote and think otherwise. That to deal with few people you need dedicated people like Sadhu Agashe who will have the licence to kill anyone, not just writing FIRs (something unworthy of the police to do, as we are made to believe).<br /><br />When TADA was repealed and the government wanted to pass newer 

In [15]:
text = dataset['test']['text'][15]
print(text)

Now, I LOVE Italian horror films. The cheesier they are, the better. However, this is not cheesy Italian. This is week-old spaghetti sauce with rotting meatballs. It is amateur hour on every level. There is no suspense, no horror, with just a few drops of blood scattered around to remind you that you are in fact watching a horror film. The "special effects" consist of the lights changing to red whenever the ghost (or whatever it was supposed to be) is around, and a string pulling bed sheets up and down. Oooh, can you feel the chills? The DVD quality is that of a VHS transfer (which actually helps the film more than hurts it). The dubbing is below even the lowest "bad Italian movie" standards and I gave it one star just because the dialogue is so hilarious! And what do we discover when she finally DOES look in the attic (in a scene that is daytime one minute and night the next)...well, I won't spoil it for anyone who really wants to see, but let's just say that it isn't very "novel"!
